In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV,LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn import metrics
import random
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from flask import Flask
import re
from sklearn.svm import SVC

In [2]:
df = pd.read_csv('manual_encoded_multiplied.csv').drop('Unnamed: 0',axis=1)

#removing space from computer science
df['education'] = df['education'].apply(lambda x: 'computer science' if x=='computer science ' else x)
df['education_label'] = df['education_label'].apply(lambda x: 1 if x==0 else x)
df['education_label'] = df['education_label'].apply(lambda x: x-1)

In [3]:
df.tail(10)

,age,education,duration,no_company_10,no_job_positions,senior,manager,linkedin link,gender_F,gender_M,education_label
612,29,other,2.0,1,2.0,0,0,https://www.linkedin.com/in/jasonzhao1/,0,1,2
613,31,other,1.0,1,1.0,0,0,https://www.linkedin.com/in/alison-herriman-54...,1,0,2
614,38,computer science,2.0,5,2.0,1,0,https://www.linkedin.com/in/surrahman/,0,1,0
615,31,other,5.0,5,6.0,0,0,NaN,0,1,2
616,47,computer science,2.0,4,1.0,0,0,https://www.linkedin.com/in/hchase0001/,0,1,0
617,40,science,0.5,2,1.0,1,0,https://www.linkedin.com/in/dave-ariens-2350b96b/,0,1,3
618,33,science,9.0,1,1.0,0,0,https://www.linkedin.com/in/tetsuro-takara-88a...,0,1,3
619,29,science,1.0,2,1.0,0,0,https://www.linkedin.com/in/jacobtsaf/,0,1,3
620,28,computer science,0.5,2,1.0,1,0,https://www.linkedin.com/in/nwoodthorpe/,0,1,0
621,40,computer science,1.0,9,1.0,1,0,https://www.linkedin.com/in/coreymartella/,0,1,0


In [4]:
def change_label_to_educ(row):
    if row == 0:
        row = 'computer science'
    elif row == 1:
        row = 'engineering'
    elif row == 2:
        row = 'other'
    else:
        row='science'
        
    return row

In [5]:
df.drop(['education','linkedin link'],axis=1,inplace=True)
df.dropna(how='any',inplace=True)

In [6]:
# def data_prepare(df):
#     df.drop(['name','current_job'], axis=1,inplace=True)
    
#     #one hot encoder for genders
#     dummies = pd.get_dummies(df[['gender']])
#     df = pd.concat([df,dummies],axis=1)
#     df.drop('gender',axis=1,inplace=True)
    
#     #label encoding education
#     label_encoder = preprocessing.LabelEncoder()
#     df['education_label'] = label_encoder.fit_transform(df['education'])
    
    
#     return df

In [7]:
def logistic_regression(df):
    
    df = df.sample(frac=1)
    #getting the x and y from the data
    x_df = df.drop(columns=['manager','senior'],axis=1)
    y_df = df[['manager']]
    #dividing into test and train
    X_train, X_test = x_df.iloc[:500], x_df.iloc[500:]
    y_train, y_test = y_df.iloc[:500], y_df.iloc[500:]
    
    #removing gender
    gender_test = x_df[['gender_F','gender_M']].iloc[500:].reset_index().drop('index',axis=1)
    education_test = x_df['education_label'].iloc[500:].reset_index().drop('index',axis=1)
    #Normalizing the data
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train_transformed = scaler.transform(X_train)
    X_test_transformed = scaler.transform(X_test)
    
    #Creating the cross validation and training
    clf = LogisticRegressionCV(cv=5, random_state=0).fit(X_train_transformed,y_train)
    
    #predicting
    y_predicted = clf.predict(X_test_transformed)
    accuracy = metrics.accuracy_score(y_test, y_predicted)
    
    #predict the probability
    y_probability = clf.predict_proba(X_test_transformed)
    
    print("The accuracy is: ", accuracy)
    
    
    #making dataframe of everything
    pre_df = pd.DataFrame(y_predicted,columns=['Manager'])
    prob_df = pd.DataFrame(y_probability,columns=['False','True'])
    final_result = pd.concat([gender_test,education_test,prob_df,pre_df],axis=1)
    
    return clf,final_result


def svm_model(df):
    
    df = df.sample(frac=1)
    #getting the x and y from the data
    x_df = df.drop(columns=['manager','senior'],axis=1)
    y_df = df[['manager']]
    #dividing into test and train
    X_train, X_test = x_df.iloc[:500], x_df.iloc[500:]
    y_train, y_test = y_df.iloc[:500], y_df.iloc[500:]
    
    #removing gender
    gender_test = x_df[['gender_F','gender_M']].iloc[500:].reset_index().drop('index',axis=1)
    education_test = x_df['education_label'].iloc[500:].reset_index().drop('index',axis=1)
    #Normalizing the data
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train_transformed = scaler.transform(X_train)
    X_test_transformed = scaler.transform(X_test)
    
    #Creating the cross validation and training
    clf = SVC(kernel='poly',degree=3,probability=True).fit(X_train_transformed,y_train)
    
    #predicting
    y_predicted = clf.predict(X_test_transformed)
    accuracy = metrics.accuracy_score(y_test, y_predicted)
    
    #predict the probability
    y_probability = clf.predict_proba(X_test_transformed)
    
    print("The accuracy is: ", accuracy)
    
    
    #making dataframe of everything
    pre_df = pd.DataFrame(y_predicted,columns=['Manager'])
    prob_df = pd.DataFrame(y_probability,columns=['False','True'])
    final_result = pd.concat([gender_test,education_test,prob_df,pre_df],axis=1)
    
    return clf,final_result


def linear_reg(df):
    
    df = df.sample(frac=1)
    #getting the x and y from the data
    x_df = df.drop(columns=['no_job_positions','senior'],axis=1)
    y_df = df[['no_job_positions']]
    #dividing into test and train
    X_train, X_test = x_df.iloc[:500], x_df.iloc[500:]
    y_train, y_test = y_df.iloc[:500], y_df.iloc[500:]
    
    #removing gender
    gender_test = x_df[['gender_F','gender_M']].iloc[500:].reset_index().drop('index',axis=1)
    education_test = x_df['education_label'].iloc[500:].reset_index().drop('index',axis=1)
    #Normalizing the data
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train_transformed = scaler.transform(X_train)
    X_test_transformed = scaler.transform(X_test)
    
    #Creating the cross validation and training
    clf = LinearRegression().fit(X_train_transformed,y_train)
    
    #predicting
    y_predicted = clf.predict(X_test_transformed)
#     accuracy = metrics.accuracy_score(y_test, y_predicted)
    
    #predict the probability
    #y_probability = clf.predict_proba(X_test_transformed)
    
#     print("The accuracy is: ", accuracy)
    
    
    #making dataframe of everything
#     pre_df = pd.DataFrame(y_predicted,columns=['Manager'])
#     prob_df = pd.DataFrame(y_probability,columns=['False','True'])
#     final_result = pd.concat([gender_test,education_test,prob_df,pre_df],axis=1)
    
    return clf

def predict(clf, age, duration, no_company, no_job, gender_F, gender_M, education):
#     if education == 'science':
#         education_label = 3
#     elif education == 'computer science':
#         education_label = 0
#     elif education == 'other':
#         education_label = 2
#     elif education == 'engineering':
#         education_label = 1
    
    x = [age,duration, no_company, no_job, gender_F, gender_M,education]
    x = np.array(x)
    y = clf.predict(x.reshape(1,-1))

    return y

def predict_linear(model, age,duration,no_company, manager,gender_f, gender_M,education):
    x=[age,duration,no_company,manager, gender_f, gender_M, education]
    x = np.array(x)
    y = model.predict(x.reshape(1,-1))
    
    return y

In [8]:
model,final_result = logistic_regression(df)
model_svm, final_result_svm = svm_model(df)
model_linear = linear_reg(df)

D:\anaconda\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


The accuracy is:  0.7058823529411765
The accuracy is:  0.8235294117647058


D:\anaconda\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [9]:
y = predict_linear(model_linear, 25,5,2,4,1,0,1)

In [10]:
print(y)

[[2.86670721]]


In [11]:
final_result_non_labeled = final_result.copy(deep=True)
final_result_non_labeled['education_label'] = final_result['education_label'].apply(change_label_to_educ)

In [12]:
final_result_non_labeled

,gender_F,gender_M,education_label,False,True,Manager
0,0,1,computer science,0.147296,0.852704,1
1,0,1,science,0.849601,0.150399,0
2,0,1,other,0.823216,0.176784,0
3,0,1,computer science,0.672302,0.327698,0
4,1,0,science,0.951460,0.048540,0
...,...,...,...,...,...,...
114,1,0,science,0.975583,0.024417,0
115,1,0,engineering,0.897445,0.102555,0
116,0,1,engineering,0.863133,0.136867,0
117,0,1,science,0.959073,0.040927,0


In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

@app.route('/manager',methods=['GET'])
def get_api_recommendation():
    x = []
    age = int(request.args.get('age'))
    duration = int(request.args.get('duration'))
    no_company = int(request.args.get('no_company'))
    no_job = int(request.args.get('no_job'))
    gender_f = int(request.args.get('gender_f'))
    gender_m = int(request.args.get('gender_m'))
    education = int(request.args.get('education'))
    x = [age,duration,no_company,no_job,gender_f,gender_m,education]
    y = predict(model,age,duration,no_company,no_job,gender_f,gender_m,education)
    return {'prediction': str(y[0])}

@app.route('/movement')
def get_api_movement():
    x = []
    age = int(request.args.get('age'))
    duration = int(request.args.get('duration'))
    no_company = int(request.args.get('no_company'))
    manager = int(request.args.get('manager'))
    gender_f = int(request.args.get('gender_f'))
    gender_m = int(request.args.get('gender_m'))
    education = int(request.args.get('education'))
    #x = [age,duration,no_company,no_job,gender_f,gender_m,education]
    y = predict_linear(model_linear,age,duration,no_company,manager,gender_f,gender_m,education)
    return {'prediction': str(y[0])}

if __name__=='__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [09/May/2021 16:29:18] "GET /movement?age=31&duration=5&no_company=4&manager=1&gender_f=0&gender_m=1&education=1 HTTP/1.1" 200 -


 * Running on http://0276a439df2a.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
